# Alternative Margin Threshold Checking

Notebook to test out my new idea for using some basic spherical trigonometry to calculate margin caches in an efficient and accurate way.

## Single Point Calculation

first, we should test out our math on a single data point. let's use a real one from one of my margin cache runs.

In [519]:
import pandas as pd
import numpy as np
import healpy as hp
from astropy.coordinates import SkyCoord

In [520]:
# data from my "margin_cache_truth" test run with the brute force margin checking
# margin_threshold == 0.1 degrees
df = pd.read_parquet("/Users/maxwest/data_sets/hipscat_import/gaia_margin_cache_truth/Norder=3/Dir=0/Npix=4.parquet")

# random test point with good astrometry (not that it really matters)
index = 20
ra, dec = df["ra"].values[index], df["dec"].values[index]
ra, dec

(56.14906022125143, 9.55430638203006)

let's take this point and get the minimum distance between it and the separations against it and all boundary points. this will serve as a good benchmark to determine how good our math is.

we should note that the value we end up calculating should be less than this as this measurement is the "hypotenuse" of our spherical triangle.

In [521]:
bounds_test = hp.vec2dir(
    hp.boundaries(2**3, 4, step=100, nest=True), lonlat=True
)

sc1 = SkyCoord(ra=ra, dec=dec, unit="deg")
sc2 = SkyCoord(ra=bounds_test[0], dec=bounds_test[1], unit="deg")

seps = sc1.separation(sc2).degree
minv = np.min(seps)
mini = np.argmin(seps)
minv, mini

(0.09858480247714595, 199)

what are the separations on either side of our minimum point? the smallest one is going to be the one to check for the margin threshold.

In [522]:
opi = 0
seps[mini-1], seps[mini+1]
if seps[mini-1] <= seps[mini+1]:
    opi = mini-1
else:
    opi = mini+1
seps[mini-1], seps[mini+1], opi

(0.13709786115228087, 0.10718199595645189, 200)

now let's get the distance between our two boundary points points

In [523]:
bp1 = SkyCoord(ra=bounds_test[0][opi], dec=bounds_test[1][opi], unit="deg")
bp2 = SkyCoord(ra=bounds_test[0][mini], dec=bounds_test[1][mini], unit="deg")
bp1.separation(bp2).degree

0.07362695789025786

- $x$ = separation between our data point and boundary point 1
- $y$ = separation between our data point and boundary point 2
- $z$ = separation between our two boundary points

In [524]:
x = seps[opi] * (np.pi / 180.)
y = seps[mini] * (np.pi / 180.)
z = bp1.separation(bp2).degree * (np.pi / 180.)
x, y, z

(0.0018706787282993344, 0.0017206293956544586, 0.0012850328334122175)

to find the the perpendicular bisector of this triangle (which is approximately the shortest distance between the data point and the healpix polygon), we will have to find

$\sin(a) = \sin(A)\sin(x)$

$a = \arcsin(\sin(A),\sin(x))$

where $a$ is the bisector, $x$ is the arc between our data point and one of the boundary points, and $A$ is the angle between that arc and the arc of the healpix boundary.

this identity comes from [Napier's rules for right spherical triangles](https://en.wikipedia.org/wiki/Spherical_trigonometry#Napier's_rules_for_right_spherical_triangles).

we already have $x$ from our separation calculations, but we're going to need to calculate the angle $A$. to do this, we can make use of the [spherical law of cosines](https://en.wikipedia.org/wiki/Spherical_law_of_cosines). let's use the identity

$\cos(y) = \cos(x)\cos(z) + \sin(x)\sin(z)\cos(A)$

from there we can solve for A with

$\sin(x)\sin(z)\cos(A) = \cos(y) - \cos(x)\cos(z)$

$\cos(A) = {\cos(y) - \cos(x)\cos(z) \over \sin(x)\sin(z)}$

$A = {\arccos({\cos(y) - \cos(x)\cos(z) \over \sin(x)\sin(z)})}$

and now we have all the information to find the bisector and compare it to our margin threshold! 

In [525]:
ang = np.arccos(
    (np.cos(y) - (np.cos(x) * np.cos(z))) / (np.sin(x) * np.sin(z))
)
ang

1.097805699243744

In [526]:
bisector = np.arcsin(np.sin(ang) * np.sin(x))
bisector * (180. / np.pi)

0.09541446117301507

and look at that, the value that we were expecting!! now let's tie this code into a nice function and run it with our $x$ and $y$ swapped (i.e., make sure that the bisector is still the same if we start from the other half of the triangle).

In [527]:
def find_perpendicular_bisector(x, y, z):
    ang = np.arccos(
        (np.cos(y) - (np.cos(x) * np.cos(z))) / (np.sin(x) * np.sin(z))
    )
    bisector = np.arcsin(np.sin(ang) * np.sin(x))
    return bisector

In [528]:
half_triangle_one = find_perpendicular_bisector(x, y, z)
half_triangle_two = find_perpendicular_bisector(y, x, z)
half_triangle_one, half_triangle_two

(0.001665296501485404, 0.0016652965014447507)

excellent!!

## Testing at Different Granularities

we tested out the math on a set of of boundaries with a `step = 100`, meaninging we had a very fine sampling of points along the boundaries of the healpixel. the hope for this function is that we can run it at much lower steps to save computation time while still getting a good result. let's rig our code up and get a spead of different possible boundary sets.

In [529]:
def margin_check_point(ra, dec, order, pixel, step):
    bounds = hp.vec2dir(
        hp.boundaries(2**order, pixel, step=step, nest=True), lonlat=True
    )

    point_coord = SkyCoord(ra=ra, dec=dec, unit="deg")
    bound_coords = SkyCoord(ra=bounds[0], dec=bounds[1], unit="deg")

    separations = point_coord.separation(bound_coords).degree
    minimum_index = np.argmin(separations)

    p_1, m_1 = minimum_index + 1, minimum_index - 1
    if minimum_index == 0:
        m_1 = len(separations) - 1
    if minimum_index == len(separations) - 1:
        p_1 = 0

    if separations[m_1] <= separations[p_1]:
        other_index = m_1
    else:
        other_index = p_1

    bc1 = SkyCoord(ra=bounds[0][minimum_index], dec=bounds[1][minimum_index], unit="deg")
    bc2 = SkyCoord(ra=bounds[0][other_index], dec=bounds[1][other_index], unit="deg")

    x = separations[minimum_index]
    y = separations[other_index]
    z = bc1.separation(bc2).degree

    return find_perpendicular_bisector(x, y, z)

In [530]:
results = []

for s in [1000, 100, 10, 5, 3, 1]:
    results.append(
        margin_check_point(ra=ra, dec=dec, order=3, pixel=4, step=s)
    )

results

[0.09541217130260496,
 0.09543372454130347,
 0.09603144150540482,
 0.09683408121708949,
 0.09907704662684182,
 0.09523052054743339]

as you can see, the more we increase our `step`, the more accurate of a measurement of our bisector we get.

## Speed Ups

now that we can confirm that the math checks out, let's try and make the calculation faster.

### Boundary Point Section Distances

above, we calculated the distance between the boundary points on the fly, but since we'll be using a consistent `step` value, we can calculate that ahead of time.

In [531]:
def get_boundary_point_distances(order, pixel, step):
    boundary_points = hp.vec2dir(
        hp.boundaries(2**order, pixel, step=step, nest=True), lonlat=True
    )

    # shift forward all the coordinates by one
    shift_ra = np.roll(boundary_points[0], 1)
    shift_dec = np.roll(boundary_points[1], 1)

    coord_set_1 = SkyCoord(ra=boundary_points[0], dec=boundary_points[1], unit="deg")
    coord_set_2 = SkyCoord(ra=shift_ra, dec=shift_dec, unit="deg")

    separations = coord_set_1.separation(coord_set_2).degree
    return separations

In [532]:
quad_lengths = get_boundary_point_distances(3,4,1)
quad_lengths

array([7.33863363, 7.33863363, 7.35465675, 7.35465675])

### Getting Separations Quickly with Numpy Optimization

in the actual code, calculating all the separations is still acting as a bottleneck. now that we have a costant value for sampling we can use some numpy tricks to hopefully speed everything up.

In [533]:
ra_array = np.array([df["ra"].values])
dec_array = np.array([df["dec"].values])

bounds_ra = np.array([bounds_test[0]])
bounds_dec = np.array([bounds_test[1]])

axis_0_repeat = len(ra_array[0])
axis_1_repeat = len(bounds_ra[0])

axis_0_repeat, axis_1_repeat

(13238, 400)

In [534]:
sc1 = SkyCoord(
    ra=np.repeat(bounds_ra, axis_0_repeat, axis=0),
    dec=np.repeat(bounds_dec, axis_0_repeat, axis=0),
    unit="deg"
)

sc2 = SkyCoord(
    ra=np.repeat(np.reshape(ra_array, (-1, 1)), axis_1_repeat, axis=1),
    dec=np.repeat(np.reshape(dec_array, (-1, 1)), axis_1_repeat, axis=1),
    unit="deg"
)

arr = sc2.separation(sc1).degree
arr

array([[9.95455254, 9.9035638 , 9.85288725, ..., 9.8055929 , 9.85491986,
        9.90457469],
       [9.95036235, 9.89938241, 9.84871488, ..., 9.8013763 , 9.85071204,
        9.90037567],
       [9.95020424, 9.89922312, 9.84855441, ..., 9.80122184, 9.85055634,
        9.90021875],
       ...,
       [0.13401494, 0.15692195, 0.20501546, ..., 0.35347697, 0.28023194,
        0.20704615],
       [0.12060942, 0.15964265, 0.21718455, ..., 0.33536777, 0.26282797,
        0.19088496],
       [0.12654865, 0.16578231, 0.22292763, ..., 0.34070734, 0.26828127,
        0.19651807]])

In [535]:
for row in arr[0:10]:
    print(np.min(row))

0.1214058789493936
0.11748261228991881
0.11755537636791677
0.11859312343716596
0.13097118741779418
0.12868051929412974
0.13496799350614985
0.11657865317457006
0.13965407629245388
0.13473137785441147
